# 5.2.4 Creación de un pipeline para clasificación de texto

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import warnings
warnings.filterwarnings("ignore")

## 5.2.4.1 Preparación de datos

In [ ]:
X_train, y_train = fetch_20newsgroups(
    subset="train",
    return_X_y=True,
)

X_test, y_test = fetch_20newsgroups(
    subset="test",
    return_X_y=True,
)

## 5.2.4.2 Especificación del pipeline

In [ ]:
pipeline = Pipeline(
    [
        # ---------------------------------------------------------------------
        # Convert a collection of text documents to a matrix of token counts.
        ("countVectorizer", CountVectorizer()),
        # ---------------------------------------------------------------------
        # Transform a count matrix to a normalized tf or tf-idf representation.
        ("tfidfTransformer", TfidfTransformer()),
        # ---------------------------------------------------------------------
        # Linear classifiers (SVM, logistic regression, etc.) with SGD training.
        ("sgdClassifier", SGDClassifier()),
    ]
)

## 5.2.4.3 Búsqueda de hiperparámetros óptimos

In [ ]:
param_grid = {
    "countVectorizer__max_df": (0.5, 0.75, 1.0),
    "sgdClassifier__max_iter": (20,),
    "sgdClassifier__alpha": (0.00001, 0.000001),
    "sgdClassifier__penalty": ("l2", "elasticnet"),
}

gridSearchCV = GridSearchCV(
    pipeline,
    param_grid,
    n_jobs=-1,
)

gridSearchCV.fit(X_train, y_train)

In [ ]:
gridSearchCV.best_score_

In [ ]:
best_parameters = gridSearchCV.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("{:>30s}: {:s}".format(param_name, repr(best_parameters[param_name])))

## 5.2.4.4 Evaluación

In [ ]:
y_pred = gridSearchCV.predict(X_test)

cm = confusion_matrix(
    y_true=y_test,
    y_pred=y_pred,
)

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=range(20)).plot()

In [ ]:
print('ok_')